In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import copy
from datetime import datetime

sns.set()

today = datetime.today()

def get_future_underlying_symbol(digit_id: str = '99'):
    df = all_instruments(type='Future', date=today)
    df2 = df.drop_duplicates(subset='underlying_symbol')
    df2 = copy(df2)
    df2['dominant'] = df2['underlying_symbol'] + digit_id
    df2 = df2[['dominant', 'exchange']]
    return df2

def get_hist_vol(symbol, start, end, n):
    name = symbol + '_hv'
    s = get_price(symbol, start, end, fields='close')
    if s is None:
        print('%s is None' % symbol)
        return
    if not s.empty:
        df = pd.DataFrame(s)
        df['returns'] = df.close.pct_change()
        df['returns'] = np.log(df['returns'] + 1)
        df[name] = df['returns'].rolling(n).std() * np.sqrt(252) * 100
        return df[name]
    
def get_symbol_by_exchange(exchange):
    df = get_future_underlying_symbol()
    df = df[df.exchange == exchange]
    return df['dominant'].values

def get_vol_stats():
    symbols = get_future_underlying_symbol()['dominant'].values
    stats_list = []
    for symbol in symbols:
        data = get_hist_vol(symbol, '2014-01-01', today, 60)
        if data is None or data.empty:
            continue
        stats = data.describe()
        stats.name = symbol
        stats_list.append(stats)
    df = pd.concat(stats_list, axis=1)
    df = df.T
    return df

def plot_vol_by_exchange(exchange):
    symbols = get_symbol_by_exchange(exchange)
    row_num = len(symbols)
    basic_width = 10
    fig = plt.figure(figsize=(basic_width, basic_width * row_num * 0.8))
    for idx in range(0, len(symbols)):
        symbol_name = symbols[idx]
        data = get_hist_vol(symbol_name, '2014-01-01', today, 60)
        if data is None or data.empty:
            continue
        plot = plt.subplot(row_num, 1, idx+1)
        data.plot(title=symbol_name)
        
def plot_vol_box_by_exchange(exchange):
    symbols = get_symbol_by_exchange(exchange)
    row_num = len(symbols)
    basic_width = 10
    fig = plt.figure(figsize=(basic_width, basic_width * row_num * 0.8))
    for idx in range(0, len(symbols)):
        symbol_name = symbols[idx]
        data = get_hist_vol(symbol_name, '2014-01-01', today, 60)
        if data is None or data.empty:
            continue
        ax = plt.subplot(row_num, 1, idx+1)
        ax.set_title(symbol_name)
        sns.boxplot(data=data, orient='v', ax=ax)
    

In [ ]:
plot_vol_by_exchange('SHFE')
plot_vol_by_exchange('DCE')
plot_vol_by_exchange('CZCE')
plot_vol_by_exchange('CFFEX')
plot_vol_by_exchange('INE')

In [ ]:
# plot_vol_box_by_exchange('SHFE')

In [ ]:
# df = get_future_underlying_symbol()
# print(df)

# df_sel = df[df.exchange=='CFFEX']
# print(df_sel)

# df = get_symbol_by_exchange('CFFEX')
# print(df)

# plot_vol_by_exchange('CFFEX')


In [ ]:

# symbol = 'CU99'
# vol = get_hist_vol(symbol, '2015-01-01', '2019-09-01', 60)
# vol.describe()
# sns.boxplot(vol, orient='v')

# vol2 = get_hist_vol('RB99', '2015-01-01', '2019-09-01', 60)
# # vol.plot(title=symbol)

# fig = plt.figure(figsize=(10, 20))
# plot_1 = plt.subplot(2, 1, 1)
# vol.plot(title=symbol)

# plot_2 = plt.subplot(2, 1, 2)
# vol2.plot(title='rb99')

df = get_vol_stats()
df.to_csv('volbox.csv')
